In [ ]:
# Function uses to download the M2E2 data from google drive onto the server 

def download_data():
    zip_file_ids = [
        (100,"1AX3NLB4MyvaTnxKv_8VwMrUlSo0177AN"),
        (200,"1bbcaf8CQQyhMsovXnBnHiW6z0xhNdaKm"),
        (300,"1xbPB4CG39PWX7_EBxguocu9yGczucABc"),
        (400,"1CUhhqQzs5tE-LYwVoYXtlRYR4Jmo30nu"),
        (500,"1Nu7WN62TNYlxzNSLFkZr35OpckYzBSqX"),
        (600,"1_dQ18MwNeMyU328CqrA3Swu1BXkIYUTM"),
        (700,"1N1P2H7yGAr39PdJwAanSj0siIYTjGtO4"),
        (800,"1yGlkuA_CAn2p8R7F4qqqj2q52JN8anxF"),
        (900,"1OhG9CXjYUxO_JiRF9O9Xj2Fw3I4saxnZ"),
        (1000,"1amrE6ATILt3Y1xxXFRbccSpbFNsyeMs5"),
        (1100,"1cgalC55A6spWM4d0k0Q3GPMaOBqqvBX9"),
        (1200,"1iDL2eHAj0q79Gi1_hrF6-pu6f0EfXae3"),
        (1300,"169mPKJS8XFLpYHibPAjx5UckQ6ctRi8U"),
        (1400,"1v9VHLG6hc6Ug3t4bK10vGmvjhhzdTnrz"),
        (1500,"1eS3E1n_TsP4glsnHi0EYbAy5uobu6XAj")
    ]
    
    for zip_file_name, zip_file_id in zip_file_ids: 
        os.system("gdown https://drive.google.com/uc?id={}".format(zip_file_id))
        os.system("unzip {}.zip".format(zip_file_name))
        os.system("rm {}.zip".format(zip_file_name))
        os.system("mv /kiwi-data/users/shoya/AIDA/m2e2_data/{}/* /kiwi-data/users/shoya/AIDA/m2e2_data".format(zip_file_name))
        os.system("rm -rf /kiwi-data/users/shoya/AIDA/m2e2_data/{}".format(zip_file_name))

In [ ]:
# Function get a csv of absolute paths for video and feature paths 
# The CSV file is then used as an input in the howto100m feature extraction

import pandas as pd 
import os 

def get_input_csv_for_feature_extraction(video_path,output_folder,dimension='2D'):
    df = pd.DataFrame()
    video_files = [s for s in os.listdir(video_path) if s.endswith('.mp4')]
    npy_file_names = [v.replace(".mp4",".npy") for v in video_files]
    
    abs_video_paths = [os.path.join(video_path,v) for v in video_files]
    abs_npy_paths = [os.path.join(output_folder,n) for n in npy_file_names]
    
    df['video_path'] = abs_video_paths 
    df['feature_path'] = abs_npy_paths 
    
    df.to_csv("video_input_paths_{}.csv".format(dimension),index=False)
    
    print("Got {} rows".format(df.shape[0]))
    
    
# get_input_csv_for_feature_extraction("/kiwi-data/users/shoya/AIDA/m2e2_clips","/kiwi-data/users/shoya/AIDA/howto100m_video_features/2d",dimension='2D')
# get_input_csv_for_feature_extraction("/kiwi-data/users/shoya/AIDA/m2e2_clips","/kiwi-data/users/shoya/AIDA/howto100m_video_features/3d",dimension='3D')
    

In [ ]:
# A function to create CSV files to be used in training script 
# For each video name, we get the corresponding file paths for the 2D and 3D video features. 

import pandas as pd 
import os 
from sklearn.model_selection import train_test_split
import json 

def format_dataset(clips_folder_path, features_folder_path,sentences_path):
    df = pd.DataFrame()
    valid_videos = list(json.load(open(sentences_path)).keys())
    video_files = [s for s in os.listdir(clips_folder_path) if s.endswith('.mp4') and s in valid_videos] 
    paths_2d = [os.path.join(features_folder_path,'2d',v.replace('.mp4','.npy')) for v in video_files]
    paths_3d = [os.path.join(features_folder_path,'3d',v.replace('.mp4','.npy')) for v in video_files]
    df['video_id'] = video_files
    df['2d'] = paths_2d
    df['3d'] = paths_3d
    
    train_df, test_df = train_test_split(df, test_size=0.2)
    
    train_df.to_csv('data_paths_train.csv',index=False)
    test_df.to_csv('data_paths_test.csv',index=False)
    
    
format_dataset("/kiwi-data/users/shoya/AIDA/m2e2_clips","/kiwi-data/users/shoya/AIDA/howto100m_video_features",'/kiwi-data/users/shoya/AIDA/event_occurences_video_and_text_pairs.json ')
    

In [53]:
import json
import os 
import re 


# Function get the corresponding sentence given a video
def get_corresponding_sentence(video_name,output_file_name,event_coreferences,video_clip_text_pair,data_dir):
    text_file = open(os.path.join(data_dir,video_name.replace('.mp4','.txt'))).read() 
    coreference_texts = [] 
    for coreference_num in event_coreferences:   
        if coreference_num > 0:
            up_to_event = text_file[:text_file.find("Event {}".format(coreference_num-1))] # OFFSET BY ONE???? 
            after_sentence = up_to_event[up_to_event.rfind("Sentence:\n") + len("Sentence:\n"):]

            next_event_start = re.search("Event [0-9]+", after_sentence)
            if next_event_start is not None:
                corefer_sentence = after_sentence[:re.search("Event [0-9]+", after_sentence).start()].strip()
            else:
                corefer_sentence = after_sentence.strip()
            coreference_texts.append(corefer_sentence)
    if coreference_texts != []:
        video_clip_text_pair[output_file_name] = coreference_texts
    return video_clip_text_pair     
 
# function to create a video clip from the whole video
def clip_video(video_name,data_dir,output_data_dir,output_file_name,clip_start,clip_end):
    os.system("ffmpeg -ss {} -i '{}' -to {} '{}'".format(clip_start,os.path.join(data_dir,video_name), clip_end-clip_start, os.path.join(output_data_dir,output_file_name)))

# a function to 1) clip videos and 2) get the video-sentence pairs saved into a JSON file    
def make_video_clips(data_dir,output_data_dir,master_json_path):
    master_json = json.load(open(master_json_path))
    total_vids = 0 
    event_cooccurences = 0 
    video_clip_text_pair = {}
    for video_name, annotations in master_json.items():
        for annotation in annotations:
            event_coreferences = set()
            for key_frame in annotation['Key_Frames']:
                for obj in key_frame['Arguments']: 
                    if obj['Event_Coreference'] is not None:
                        event_coreferences.add(obj['Event_Coreference'])
                    
            if len(event_coreferences) > 0:
                clip_start, clip_end = annotation['Temporal_Boundary'][0],annotation['Temporal_Boundary'][1]
                output_file_name = video_name.replace('.mp4','') + ' clipped_{}_{}.mp4'.format(clip_start,clip_end)
                print(output_file_name)
                video_clip_text_pair = get_corresponding_sentence(video_name,output_file_name,event_coreferences,video_clip_text_pair,data_dir)
                clip_video(video_name,data_dir,output_data_dir,output_file_name,clip_start,clip_end)
                event_cooccurences += 1 
                
        total_vids += 1 
                
    print("{} total videos, {} co-occurences".format(total_vids,event_cooccurences))
    
    # save mapping 
    with open('event_occurences_video_and_text_pairs.json', 'w') as f:
        json.dump(video_clip_text_pair, f)


In [1]:
make_video_clips('/Users/shoya.yoshida/AIDA/data_labeling/data/1200','/Users/shoya.yoshida/AIDA/Coreference/clipped_output','/Users/shoya.yoshida/AIDA/data_labeling/master.json')

# make_video_clips('/kiwi-data/users/shoya/AIDA/m2e2_data','/kiwi-data/users/shoya/AIDA/m2e2_clips','/kiwi-data/users/shoya/AIDA/master.json')



In [18]:
master_json = json.load(open('/Users/shoya.yoshida/AIDA/data_labeling/master.json'))
# master_json

In [19]:
len(master_json.keys())

860